## 1. Optimización de Hiperparámetros

In [1]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow as tf

In [125]:

observaciones = 1000

x1 = np.random.uniform(low = -10, high = 10, size = (observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

X = np.column_stack((x1,x2))



In [126]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

y = 2 * x1 - 3 * x2 + 5 + ruido

In [127]:
np.savez('Datos_TF', entradas = X, targets = y)

In [128]:
# Se cargan los datos desde el archivo NPZ
datos = np.load('Datos_TF.npz')

In [129]:
tamanio_entrada = 2

tamanio_salida = 1

In [130]:
modelo = tf.keras.Sequential([
                           
                            tf.keras.layers.Dense(tamanio_salida,
                                                 kernel_initializer = tf.random_uniform_initializer(minval = -0.1, maxval = 0.1),
                                                 bias_initializer = tf.random_uniform_initializer(minval = -0.1, maxval = 0.1)
                                                 )
                            ])

In [131]:
optimizador_adhoc = tf.keras.optimizers.SGD(learning_rate = 0.02)

In [132]:
loss = tf.keras.losses.Huber()
modelo.compile(optimizer = optimizador_adhoc, loss=loss)

In [137]:
modelo.fit(datos['entradas'], datos['targets'], epochs = 100, verbose = 0)

In [134]:
modelo.layers[0].get_weights()    # el cero (0) es porque solo tenemos una capa

[array([[ 1.9892693],
        [-2.9992857]], dtype=float32),
 array([4.967491], dtype=float32)]

In [97]:
pesos = modelo.layers[0].get_weights()[0]
pesos

array([[ 1.9455736],
       [-2.925254 ]], dtype=float32)

In [28]:

sesgos = modelo.layers[0].get_weights()[1]
sesgos

array([4.996776], dtype=float32)

In [135]:
modelo.predict_on_batch(datos['entradas']).round(1)[:5, :]

array([[18. ],
       [38.7],
       [13.7],
       [20.7],
       [ 9.4]], dtype=float32)

In [136]:
datos['targets'].round(1)[:5, :]

array([[18.1],
       [38.4],
       [12.9],
       [20.6],
       [ 8.9]])

In [138]:
fig = px.scatter(x = np.squeeze(modelo.predict_on_batch(datos['entradas'])), 
                 y =  np.squeeze(datos['targets']))

fig.update_layout(
    title = "Comparación predicciones vrs metas",
    xaxis_title = "Predicciones",
    yaxis_title = "Metas",
    width = 600,
    height = 400,)

fig.show()

## 2. Validación cruzada

In [ ]:
from sklearn.model_selection import cross_validate
from scikeras.wrappers import KerasRegressor

model = KerasRegressor(model=modelo, epochs=100, verbose=0, )
cv_results = cross_validate(model, X=datos['entradas'], y=datos['targets'], cv=10, scoring=["neg_root_mean_squared_error"])
cv_results
